<a href="https://colab.research.google.com/github/ZJouba/Advent-of-Code-2022/blob/main/Day9/Day9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import numpy as np

file = requests.get('https://raw.githubusercontent.com/ZJouba/Advent-of-Code-2022/main/Day8/Day8.txt')
input = list(filter(None, file.text.split('\n')))
input = np.asarray([list(tree) for tree in list(filter(None, file.text.split('\n')))], dtype=np.int32)

# file = '''30373
# 25512
# 65332
# 33549
# 35390'''

# input = np.asarray([list(tree) for tree in list(filter(None, file.split('\n')))], dtype=np.int32)

border = (input.shape[0] + input.shape[1]) * 2 - 4

print(input)

[[1 1 2 ... 2 2 2]
 [0 0 2 ... 1 0 2]
 [0 0 2 ... 1 2 1]
 ...
 [2 0 1 ... 2 2 2]
 [0 0 1 ... 1 1 0]
 [0 1 2 ... 2 2 1]]


In [ ]:
shape = input.shape

total = 0
for idx, tree in np.ndenumerate(input):
  if not any(loc == 0 or loc == shape[0]-1 or loc == shape[1]-1 for loc in idx):
    if all(tree > neighbor for neighbor in input[idx[0], :idx[1]]) or \
      all(tree > neighbor for neighbor in input[:idx[0], idx[1]]) or \
      all(tree > neighbor for neighbor in input[idx[0], idx[1]+1:]) or \
      all(tree > neighbor for neighbor in input[idx[0]+1:, idx[1]]):
      total += 1

print(total + border)


1827


In [ ]:
shape = input.shape

scenicScores = []
for idx, tree in np.ndenumerate(input):
  if not any(loc == 0 or loc == shape[0]-1 or loc == shape[1]-1 for loc in idx):
    score = []
    left = list(reversed(list(tree > neighbor for neighbor in input[idx[0], :idx[1]])))
    leftScore = 0
    for i in left:
      leftScore += 1
      if not i: break
    if leftScore > 0: score.append(leftScore)

    top = list(reversed(list(tree > neighbor for neighbor in input[:idx[0], idx[1]])))
    topScore = 0
    for i in top:
      topScore += 1
      if not i: break      
    if topScore > 0: score.append(topScore)

    right = list(tree > neighbor for neighbor in input[idx[0], idx[1]+1:])
    rightScore = 0
    for i in right:
      rightScore += 1
      if not i: break      
    if rightScore > 0: score.append(rightScore)

    down = list(tree > neighbor for neighbor in input[idx[0]+1:, idx[1]])
    downScore = 0
    for i in down:
      downScore += 1
      if not i: break      
    if downScore > 0: score.append(downScore)

    totalScenic = np.prod(score)
    # print(f'Tree {idx} score = {totalScenic}')
    scenicScores.append(totalScenic)

print(max(scenicScores))


335580
